In [1]:
import os
os.chdir('..')
print(os.getcwd())

/home/shugo/workspace/rendering


In [2]:
import plotly.graph_objects as go


def add_mesh(fig, mesh, **kwargs):
    # fig = go.Figure()
    x, y, z = mesh.vertices.T
    i, j, k = mesh.faces.T
    fig.add_trace(
        go.Mesh3d(x=x,y=y,z=z,i=i,j=j,k=k, **kwargs)
    )
    return fig

In [45]:
import torch
import trimesh
import numpy as np

def get_another_obj_mesh(anno, name):
    obj_filename = f'data/models/{name}/textured_simple.obj'
    obj_trimesh = trimesh.load_mesh(obj_filename)

    obj_verts = torch.tensor(obj_trimesh.vertices, dtype=torch.float32)
    obj_faces = torch.tensor(obj_trimesh.faces, dtype=torch.int64)
    obj_uv = torch.tensor(obj_trimesh.visual.uv, dtype=torch.float32)
    obj_map = torch.from_numpy(np.array(obj_trimesh.visual.material.image, dtype=np.float32)) / 255.0

    return obj_verts, obj_faces, obj_uv, obj_map



In [64]:
from utils.mano import ManoLayer
from utils.utils import load_ho_meta, get_obj_mesh, apply_transform_to_mesh
import trimesh


mano = ManoLayer()
anno = load_ho_meta("data/HO3D_v3/train/ABF10/meta/0000.pkl")

mano_hand = mano(anno)

obj_verts, obj_faces, obj_uv, obj_map = get_another_obj_mesh(anno, "025_mug")
obj_verts = apply_transform_to_mesh(obj_verts, anno)

handjoints = anno['handJoints3D']

# print(mano_hand.vertices.shape, mano.faces.shape)
mano_mesh = trimesh.Trimesh(vertices=mano_hand.vertices[0], faces=mano.faces, )
obj_mesh = trimesh.Trimesh(vertices=obj_verts, faces=obj_faces)
# scene = trimesh.Scene([mano_mesh, obj_mesh])
# lights, transforms = trimesh.scene.lighting.autolight(scene)

# scene.show()

/home/shugo/workspace/rendering/utils/utils.py:18: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



torch.Size([16762, 3]) torch.Size([3, 3]) torch.Size([3])


In [56]:
import numpy as np
import torch

def calc_face_normals(verts, faces):
    v1 = verts[faces[:,0].type(torch.long)]
    v2 = verts[faces[:,1].type(torch.long)]
    v3 = verts[faces[:,2].type(torch.long)]
    face_normals = np.cross(v2-v1, v3-v1)
    face_normals = face_normals / np.linalg.norm(face_normals, axis=1)[:,None]
    return face_normals

print(calc_face_normals(obj_verts, obj_faces))
print(obj_mesh.face_normals)

[[ 0.5212089  -0.6450417  -0.55880445]
 [ 0.52541053 -0.65679204 -0.5408956 ]
 [-0.07026267 -0.9152369   0.3967425 ]
 ...
 [-0.03539494  0.9872169   0.1554026 ]
 [-0.0539282   0.99665314  0.06143529]
 [-0.0636517   0.9954907   0.07033282]]
[[ 0.52120888 -0.64504175 -0.55880448]
 [ 0.52541049 -0.65679204 -0.54089558]
 [-0.07026267 -0.91523689  0.39674248]
 ...
 [-0.03539494  0.98721691  0.15540259]
 [-0.05392821  0.99665313  0.06143529]
 [-0.0636517   0.99549071  0.07033282]]


In [30]:
def calc_vertex_normals(verts, faces):
    face_normals = calc_face_normals(verts, faces)
    vertex_normals = np.zeros(verts.shape)
    for i, face in enumerate(faces):
        vertex_normals[face] += face_normals[i]
    vertex_normals = vertex_normals / np.linalg.norm(vertex_normals, axis=1)[:,None]
    return vertex_normals

print(calc_vertex_normals(obj_verts, obj_faces))
print(obj_mesh.vertex_normals)

[[-0.20563468 -0.19256703 -0.95949587]
 [-0.87620358 -0.25694644 -0.40773253]
 [-0.86817425 -0.30585594 -0.39080125]
 ...
 [ 0.82136422  0.17140809  0.54404053]
 [ 0.8192825   0.1688788   0.54795632]
 [ 0.82258504  0.16243595  0.54494808]]
[[-0.22926027 -0.18466345 -0.95568778]
 [-0.87681972 -0.25046235 -0.41043366]
 [-0.87841527 -0.2800505  -0.38724454]
 ...
 [ 0.82155303  0.17137025  0.54376729]
 [ 0.81919873  0.16856341  0.54817863]
 [ 0.82262227  0.16219697  0.54496307]]


In [80]:
class SinedDistanceNormals:
    def __init__(self):
        pass

    def calc_sd_normals(self, obj_mesh, handjoints):
        closest, distance, triangle_id = trimesh.proximity.closest_point(obj_mesh, handjoints)
        
        obj_verts = torch.tensor(obj_mesh.vertices)
        obj_faces = torch.tensor(obj_mesh.faces)
        face_normals = calc_face_normals(obj_verts, obj_faces)
        closest_normals = face_normals[triangle_id]
        sd_normals = closest_normals * distance[:,None]

        self.closest = closest
        self.distance = distance
        self.closest_normals = closest_normals
        self._sd_normals = sd_normals
        return sd_normals

sd = SinedDistanceNormals()
sd_normals = sd.calc_sd_normals(obj_mesh, handjoints)

In [81]:
fig = go.Figure()
add_mesh(fig, mano_mesh, opacity=1.0)
add_mesh(fig, obj_mesh, opacity=0.5)
fig.add_trace(go.Scatter3d(x=handjoints[:,0], y=handjoints[:,1], z=handjoints[:,2], mode='markers', marker=dict(size=5), name='handjoints'))
fig.add_trace(
    go.Scatter3d(
        x=sd.closest[:,0], 
        y=sd.closest[:,1], z=sd.closest[:,2], mode='markers', marker=dict(size=5)
        )
    )

for p1, p2 in zip(handjoints, sd.closest):
    fig.add_trace(
        go.Scatter3d(
            x=[p1[0], p2[0]], y=[p1[1], p2[1]], z=[p1[2], p2[2]], 
            mode='lines', line=dict(width=3, color='orange'),
            name='closest'
        )
        )

for p, n in zip(sd.closest, sd_normals):
    fig.add_trace(
        go.Scatter3d(
            x=[p[0], p[0]+n[0]], y=[p[1], p[1]+n[1]], z=[p[2], p[2]+n[2]], 
            mode='lines', line=dict(width=3, color='cyan'),
            name='sd_normals'
        )
        )

# fig.add_trace(
#     go.Cone(
#         x=closest[:,0], y=closest[:,1], z=closest[:,2], u=sd_normals[:,0], v=sd_normals[:,1], w=sd_normals[:,2], colorscale='Viridis', sizemode='scaled', sizeref=10
#     )
# )
fig.show()